# 13. 持久化 - shelve

# shelve
- Python 对象持久化
- 是一个持久化工具
- “Shelf”是一种持久化的类似字典的对象
- Shelf 中的值（不是键！）实际上可以为任意 Python 对象，即 pickle 模块能够处理的任何东西
    - 包括大部分类实例、递归数据类型，以及包含大量共享子对象的对象
    - 键则为普通的字符串
- 需要导入

In [22]:
import shelve

## open()
- open()：打开一个持久化字典
- 格式：shelve.open(filename, flag='c', protocol=None, writeback=False)
    -  filename 指定下层数据库的基准文件名
        - 作为附带效果，会为 filename 添加一个扩展名并且可能创建更多的文件
        - 默认情况下，下层数据库会以读写模式打开
    - flag：可选参数，可以是：

|值|意义|
|:-:|:-:|
|r|以只读方式打开现有数据库（默认|
|w|以读写方式打开现有数据库|
|c|以读写方式打开数据库，如果不存在则创建它|
|n|始终创建一个新的空数据库，以读写方式打开|

- 
    - protocol：形参，来指定 pickle 协议版本
    - 如果可选的 writeback 形参设为 True，则所有被访问的条目都将在内存中被缓存，并会在 sync() 和 close() 时被写入
        - 这可以使得对持久化字典中可变条目的修改更方便
        - 但是如果访问的条目很多，这会消耗大量内存作为缓存，并会使得关闭操作变得非常缓慢
        - 因为所有被访问的条目都需要写回到字典（无法确定被访问的条目中哪个是可变的，也无法确定哪个被实际修改了）

## close()
- close()：同步并关闭持久化 dict 对象，对已关闭 Shelf 的操作将失败并引发 ValueError

In [23]:
# 使用shelve创建文件并使用

# 打开文件
# 此时，shv相当于一个字典
shv = shelve.open(r'shelve测试.db')

shv['one'] = 1
shv['two'] = 2
shv['three'] = 3

shv.close()

# 通过以上示例发现，shelve自动创建的不仅仅是一个shelve测试.db文件，还包括其他格式文件

In [24]:
# shelve读取 示例

shv = shelve.open(r'shelve测试.db')

print(shv['one'])
print(shv['three'])

shv.close()

1
3


In [25]:
# shelve读取 示例2

shv = shelve.open(r'shelve测试.db')

try:
    print(shv['one'])
    print(shv['dasjhdk']) # 为了防止此处报错而无法关闭Shelf，所以用try...finally语句

except Exception as e:
    print('报毛线错啊！')

finally:
    shv.close()

1
报毛线错啊！


## shelve特性
- 不支持多个应用并行写入
    - 为了解决这个问题，open的时候可以使用 flag=r
- 写回问题
    - shelve默认情况下不会对持久化对象进行任何修改
    - 解决方法： 强制写回：writeback=True

In [26]:
# shelve 只读打开
shv = shelve.open(r'shelve测试.db', flag='r')

try:
    k1 = shv['one']
    print(k1)

except Exception as e:
    print('报毛线错啊！')

finally:
    shv.close()

1


In [27]:
shv = shelve.open(r'shelve测试.db')
try:
    shv['one'] = {"eins":1, "zwei":2, "drei":3}

except Exception as e:
    print('报毛线错啊！')

finally:
    shv.close()


shv = shelve.open(r'shelve测试.db')
try:
    one = shv['one']
    print(one)

except Exception as e:
    print('报毛线错啊！')

finally:
    shv.close()

{'eins': 1, 'zwei': 2, 'drei': 3}


In [28]:
# 当shelve忘记写回，需要使用强制写回
# 错误示例

shv = shelve.open(r'shelve测试.db')
try:
    k1 = shv['one']
    print(k1)
    # 此时，一旦shelve关闭，则内容还是存在于内存中，没有写回数据库
    k1["eins"] =100

except Exception as e:
    print('报毛线错啊！')

finally:
    shv.close()


shv = shelve.open(r'shelve测试.db')
try:
    k1 = shv['one']
    # 可以看出，"eins"的内容没有变化，没有写回数据库
    print(k1)

except Exception as e:
    print('报毛线错啊！')

finally:
    shv.close()

{'eins': 1, 'zwei': 2, 'drei': 3}
{'eins': 1, 'zwei': 2, 'drei': 3}


In [29]:
# 当shelve忘记写回，需要使用强制写回
# 正确示例

shv = shelve.open(r'shelve测试.db', writeback=True) # 此处需要更改
try:
    k1 = shv['one']
    print(k1)
    # 此时，一旦shelve关闭，则内容还是存在于内存中，没有写回数据库
    k1["eins"] =100

except Exception as e:
    print('报毛线错啊！')

finally:
    shv.close()


shv = shelve.open(r'shelve测试.db')
try:
    k1 = shv['one']
    # 可以看出，"eins"的内容没有变化，没有写回数据库
    print(k1)

except Exception as e:
    print('报毛线错啊！')

finally:
    shv.close()

{'eins': 1, 'zwei': 2, 'drei': 3}
{'eins': 100, 'zwei': 2, 'drei': 3}


In [30]:
# shelve 使用 with语句 管理上下文环境

with shelve.open(r'shelve测试.db', writeback= True) as shv:
    k1 = shv['one']
    print(k1)
    k1['zwei'] = 2000

with shelve.open(r'shelve测试.db', writeback= True) as shv:
    print(shv['one'])

{'eins': 100, 'zwei': 2, 'drei': 3}
{'eins': 100, 'zwei': 2000, 'drei': 3}
